Notebook to benchmark the classification ability of the HBM of Plebani et al. 2022 [1] on a test set of CRISM data.

The HBM model training data can be found [here](http://cs.iupui.edu/~mdundar/CRISM.htm) under CRISM_labeled_pixels_ratioed.mat and should be saved in the data/CRISM_ML directory.

The HBM model expects ratioed pixel data, so need to ratio the test set. The scripts/bland_pixel_finder.py script should be used to find and collate bland pixels for the entire dataset. Please run that script before running this notebook.

References:  
1. https://github.com/Banus/crism_ml/tree/master
2. Bultel B, Quantin C, Lozac’h L. Description of CoTCAT (Complement to CRISM Analysis Toolkit). IEEE Journal of Selected Topics in Applied Earth Observations and Remote Sensing. 2015 Jun;8(6):3039–49. 

In [1]:
import scipy.io
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
import os

from crism_ml.lab import relabel, ALIASES_TRAIN, CLASS_NAMES
from crism_ml.preprocessing import remove_spikes
from crism_ml.train import train_model, iteration_weights, feat_masks, compute_scores, filter_predictions

from n2n4m.wavelengths import PLEBANI_WAVELENGTHS
from n2n4m.utils import convert_coordinates_to_xy, convert_xy_to_coordinates, label_list_to_string, label_string_to_list
import n2n4m.preprocessing as preprocessing
from n2n4m.preprocessing import LABEL_COLS

In [2]:
PACKAGE_DIR = os.path.dirname(os.path.dirname(os.getcwd()))
DATA_DIR = os.path.join(PACKAGE_DIR, 'data')


NUM_BLAND_PIXELS = 150_000
# These 4 classes were deliberately withheld from the N2N4M training dataset. 
# Therefore need to remove them from the test dataset, as the classifier won't have been trained on any samples of them
UNSEEN_PIXEL_CLASSES = [32, 23, 24, 10] 

To make it a fair test of classification ability, will use the N2N4M test set. We need to remove this data from the training set for the classifier.

In [3]:
# Load the original HBM training data
mat = scipy.io.loadmat(os.path.join(DATA_DIR, "CRISM_ML", "CRISM_labeled_pixels_ratioed.mat"))
spectra = mat["pixspec"]
coordinates = mat["pixcrds"].tolist()
im_nums = mat["pixims"]
class_labels = mat["pixlabs"]
image_names = mat["im_names"]

In [4]:
# Convert to a DataFrame for easy manipulation
df = pd.DataFrame(spectra, columns=PLEBANI_WAVELENGTHS)
df["Image_Number"] = im_nums
df["Coordinates"] = coordinates
df["Pixel_Class"] = class_labels
df

,1.021,1.02755,1.0341,1.04065,1.0472,1.05375,1.0603,1.06685,1.07341,1.07996,...,3.43701,3.44366,3.45031,3.45696,3.46361,3.47026,3.47692,Image_Number,Coordinates,Pixel_Class
0,1.157633,1.163859,1.164394,1.171796,1.167980,1.176179,1.172649,1.172700,1.173427,1.174842,...,1.080833,1.092634,1.100177,1.094970,1.093435,1.091317,1.088656,1,"[275, 3]",9
1,1.095404,1.095902,1.098150,1.099153,1.095909,1.100634,1.095856,1.097055,1.099711,1.095848,...,1.039531,1.056432,1.070553,1.072255,1.074990,1.077480,1.068430,1,"[276, 4]",9
2,1.057313,1.056104,1.057158,1.057873,1.054683,1.055796,1.055635,1.057790,1.054455,1.056135,...,1.027476,1.030738,1.032081,1.031724,1.025415,1.019628,1.014530,1,"[272, 5]",9
3,1.106403,1.104935,1.109509,1.104594,1.107657,1.105852,1.104506,1.103828,1.104012,1.103454,...,1.055920,1.057920,1.058732,1.063122,1.066273,1.061006,1.062589,1,"[273, 5]",9
4,1.064772,1.064418,1.069211,1.067394,1.072046,1.071638,1.070746,1.071261,1.074613,1.068183,...,1.023246,1.026631,1.026660,1.028470,1.032170,1.030396,1.029476,1,"[278, 5]",9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592408,0.842000,0.840261,0.839757,0.839124,0.841455,0.841077,0.840902,0.840742,0.841271,0.841928,...,0.724899,0.725529,0.726208,0.727316,0.728769,0.728942,0.729944,70,"[0, 0]",39
592409,0.869521,0.868297,0.868078,0.870621,0.871579,0.871579,0.870478,0.870124,0.870120,0.869855,...,0.796355,0.798434,0.800484,0.800654,0.799571,0.798537,0.797940,70,"[0, 0]",39
592410,0.889677,0.889782,0.889329,0.889648,0.889586,0.889381,0.887838,0.887221,0.887221,0.887226,...,0.815944,0.817836,0.818277,0.818249,0.816804,0.816804,0.815163,70,"[0, 0]",39
592411,0.932668,0.932397,0.932575,0.932980,0.932628,0.932525,0.930847,0.928193,0.929618,0.928149,...,0.778314,0.776318,0.775282,0.776606,0.778779,0.778943,0.778850,70,"[0, 0]",39


In [5]:
# Replace Image_Number with Image_Name
df["Image_Name"] = df["Image_Number"].apply(lambda x: image_names[x-1]) # Oriignally 1-indexed in MATLAB
df["Image_Name"] = df["Image_Name"].astype(str)
df = df.drop(columns=["Image_Number"])
df

,1.021,1.02755,1.0341,1.04065,1.0472,1.05375,1.0603,1.06685,1.07341,1.07996,...,3.43701,3.44366,3.45031,3.45696,3.46361,3.47026,3.47692,Coordinates,Pixel_Class,Image_Name
0,1.157633,1.163859,1.164394,1.171796,1.167980,1.176179,1.172649,1.172700,1.173427,1.174842,...,1.080833,1.092634,1.100177,1.094970,1.093435,1.091317,1.088656,"[275, 3]",9,027E2
1,1.095404,1.095902,1.098150,1.099153,1.095909,1.100634,1.095856,1.097055,1.099711,1.095848,...,1.039531,1.056432,1.070553,1.072255,1.074990,1.077480,1.068430,"[276, 4]",9,027E2
2,1.057313,1.056104,1.057158,1.057873,1.054683,1.055796,1.055635,1.057790,1.054455,1.056135,...,1.027476,1.030738,1.032081,1.031724,1.025415,1.019628,1.014530,"[272, 5]",9,027E2
3,1.106403,1.104935,1.109509,1.104594,1.107657,1.105852,1.104506,1.103828,1.104012,1.103454,...,1.055920,1.057920,1.058732,1.063122,1.066273,1.061006,1.062589,"[273, 5]",9,027E2
4,1.064772,1.064418,1.069211,1.067394,1.072046,1.071638,1.070746,1.071261,1.074613,1.068183,...,1.023246,1.026631,1.026660,1.028470,1.032170,1.030396,1.029476,"[278, 5]",9,027E2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592408,0.842000,0.840261,0.839757,0.839124,0.841455,0.841077,0.840902,0.840742,0.841271,0.841928,...,0.724899,0.725529,0.726208,0.727316,0.728769,0.728942,0.729944,"[0, 0]",39,21B59
592409,0.869521,0.868297,0.868078,0.870621,0.871579,0.871579,0.870478,0.870124,0.870120,0.869855,...,0.796355,0.798434,0.800484,0.800654,0.799571,0.798537,0.797940,"[0, 0]",39,21B59
592410,0.889677,0.889782,0.889329,0.889648,0.889586,0.889381,0.887838,0.887221,0.887221,0.887226,...,0.815944,0.817836,0.818277,0.818249,0.816804,0.816804,0.815163,"[0, 0]",39,21B59
592411,0.932668,0.932397,0.932575,0.932980,0.932628,0.932525,0.930847,0.928193,0.929618,0.928149,...,0.778314,0.776318,0.775282,0.776606,0.778779,0.778943,0.778850,"[0, 0]",39,21B59


In [6]:
# Remove any pixels with [0,0] coordinates, as these are not valid. (Multiple pixels in an image assigned to the same coordinates, and Matlab uses 1-indexing)
df = convert_coordinates_to_xy(df)
df = df[(df["x"] > 0 ) & (df["y"] > 0)]
df = convert_xy_to_coordinates(df)

In [7]:
# Now split the data into training and testing sets
train_set, test_set = preprocessing.train_test_split(df, bland_pixels=True)

In [8]:
# Now we want to save the training set as a .mat file for the CRISM_ML HBM model to use for training.
train_spectra = train_set.drop(columns=LABEL_COLS).to_numpy()
train_labels = train_set["Pixel_Class"].to_numpy()
train_im_names = train_set["Image_Name"].to_numpy()
train_coords = train_set["Coordinates"].to_numpy()
train_im_nums = [np.where(mat["im_names"] == im_name)[0]+1 for im_name in train_im_names] # Map the image names to the original image numbers
train_dict = {"pixspec": train_spectra, "pixlabs": train_labels, "pixcrds": train_coords, "pixims": train_im_nums, "im_names": image_names}

In [9]:
scipy.io.savemat("CRISM_labeled_pixels_ratioed.mat", train_dict) # Save the training set as a .mat file. Has to have the same name, so will save it to this local dir.

Data Preprocessing

In [10]:
# Read the data.
dataset = os.path.join(DATA_DIR, "extracted_mineral_pixel_data", "mineral_pixel_dataset.json")
dataset = preprocessing.load_dataset(dataset)

dataset = preprocessing.expand_dataset(dataset)
dataset = preprocessing.drop_bad_bands(dataset, bands_to_keep=PLEBANI_WAVELENGTHS)
dataset = preprocessing.impute_bad_values(dataset, threshold=1)
dataset = preprocessing.impute_atmospheric_artefacts(dataset, wavelengths=PLEBANI_WAVELENGTHS) 
noise_dataset = preprocessing.generate_noisy_pixels(dataset.iloc[:,3:], random_seed=42)
input_target_dataset = pd.concat([dataset, noise_dataset], axis=1)
train_set, test_set = preprocessing.train_test_split(input_target_dataset)

In [11]:
# Drop unseen pixel classes from the test set
test_set = label_list_to_string(test_set)
test_set = test_set[~test_set["Pixel_Class"].isin(UNSEEN_PIXEL_CLASSES)]
test_set = label_string_to_list(test_set)

/raid/home/rp1818/N2N4M/N2N4M/n2n4m/utils.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["Pixel_Class"] = dataset["Pixel_Class"].apply(lambda x: x[0])


Get bland pixels so we can ratio the data. This process enhances unusual spectral features in the data, which is useful for identifying composition.

In [12]:
# Load the bland pixel data that was found for every pixel in the dataset
bland_pixels = pd.read_json(os.path.join(DATA_DIR, "extracted_mineral_pixel_data", "bland_pixels_to_match_mineral_pixels.json"))

In [13]:
test_set = convert_coordinates_to_xy(test_set)
bland_pixels["x"] = bland_pixels["Mineral_Pixel_Coordinates"].apply(lambda x: x[0])
bland_pixels["y"] = bland_pixels["Mineral_Pixel_Coordinates"].apply(lambda x: x[1])

In [14]:
# Suitable bland pixels were not found for the entire test set, so we will restrict the test set to those with bland pixels.
lookup1 = test_set.set_index(["x", "y", "Image_Name"]).index
lookup2 = bland_pixels.set_index(["x", "y", "Image_Name"]).index
restricted_test_set = test_set[lookup1.isin(lookup2)]
restricted_test_set = convert_xy_to_coordinates(restricted_test_set)
restricted_bland_test_set = bland_pixels[lookup2.isin(lookup1)] # Also drop the blands which don't have corresponding pixels in the test set.
restricted_bland_test_set = convert_xy_to_coordinates(restricted_bland_test_set)

/raid/home/rp1818/N2N4M/N2N4M/n2n4m/utils.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["Coordinates"] = dataset.apply(lambda x: [x["x"], x["y"]], axis=1)
/raid/home/rp1818/N2N4M/N2N4M/n2n4m/utils.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["Coordinates"] = dataset.apply(lambda x: [x["x"], x["y"]], axis=1)


In [15]:
# The HBM model doesn't use the fully specific label set, they use a slightly generalised set, so convert the labels to that.
label_copy = restricted_test_set["Pixel_Class"].copy()
restricted_test_set["Pixel_Class"] = relabel(label_copy, ALIASES_TRAIN)

In [16]:
# Change some column names so they match between datasets
restricted_bland_test_set["Spectrum"] = restricted_bland_test_set["Average_Spectra"]
restricted_bland_test_set["Coordinates"] = restricted_bland_test_set["Mineral_Pixel_Coordinates"]
restricted_bland_test_set = preprocessing.expand_dataset(restricted_bland_test_set, bands=PLEBANI_WAVELENGTHS)
restricted_bland_test_set = restricted_bland_test_set.drop(columns=["Bland_Pixel_Coordinates", "Average_Spectra", "Mineral_Pixel_Coordinates"])

In [17]:
# Sort the blands and test set by image name and coordinates so that we know they match up.
restricted_test_set = convert_coordinates_to_xy(restricted_test_set)
restricted_bland_test_set = convert_coordinates_to_xy(restricted_bland_test_set)
restricted_test_set = restricted_test_set.sort_values(by=["Image_Name", "x", "y"])
restricted_bland_test_set = restricted_bland_test_set.sort_values(by=["Image_Name", "x", "y"])
restricted_test_set = restricted_test_set.reset_index(drop=True)
restricted_bland_test_set = restricted_bland_test_set.reset_index(drop=True)
restricted_test_set = convert_xy_to_coordinates(restricted_test_set)
restricted_bland_test_set = convert_xy_to_coordinates(restricted_bland_test_set)

In [18]:
# Split the high and low noise spectra apart
bland_spectra = restricted_bland_test_set.drop(columns=["Image_Name", "Coordinates"]).to_numpy()
noisy_spectra, low_noise_spectra, ancillary_data = preprocessing.split_features_targets_anciliary(restricted_test_set)

In [19]:
# Get the labels and convert to ints
restricted_labels = ancillary_data["Pixel_Class"].to_numpy()
restricted_labels = np.array([int(x[0]) for x in restricted_labels])

In [20]:
noisy_spectra = noisy_spectra.to_numpy()
low_noise_spectra = low_noise_spectra.to_numpy()

In [21]:
# Ratio the spectra
noisy_spectra_ratioed = noisy_spectra / bland_spectra
low_noise_spectra_ratioed = low_noise_spectra / bland_spectra

Train the HBM classifier

In [22]:
fin0, fin = feat_masks()
models = train_model(os.getcwd(), fin)
weights = iteration_weights(models[0].classes)

Test the HBM classifier on high-quality, low-noise data.

In [23]:
despiked_ln_spectra = remove_spikes(low_noise_spectra_ratioed) # Final despike
ln_scores = compute_scores(despiked_ln_spectra, (models, fin), weights)

In [24]:
filtered_ln_preds, ln_preds, ln_pred_probs = filter_predictions(ln_scores, models[0].classes, merge_clays=False, kls_thr=(0.5,0.7))

In [25]:
def get_classification_report(predictions, labels):
    """
    Function to generate a classification report for a set of predictions.
    Uses the CLASS_NAMES defined in crism_ml.lab.

    Parameters
    ----------
    predictions: np.ndarray
        Predicted class labels. Shape (n_samples,).
    labels: np.ndarray
        True class labels. Shape (n_samples,).

    Returns
    -------
    report: dict
        Sklearn classification report.
    """
    true_unique_classes = np.unique(labels)
    predicted_unique_classes = np.unique(predictions)
    classes_to_calculate_f1 = np.unique(np.concatenate((true_unique_classes, predicted_unique_classes)))
    string_classes = np.array([CLASS_NAMES[obs_class] for obs_class in classes_to_calculate_f1])

    report = classification_report(labels, predictions, target_names=string_classes, zero_division=0, output_dict=True)
    return report

In [27]:
class_report = get_classification_report(ln_preds, restricted_labels)
print(f"Model Accuracy: {class_report['accuracy']}")
print(f"Model F1 Score: {class_report['macro avg']['f1-score']}")
print(f"Model Precision: {class_report['macro avg']['precision']}")
print(f"Model Recall: {class_report['macro avg']['recall']}")

Model Accuracy: 0.7480317652018367
Model F1 Score: 0.2082495252817787
Model Precision: 0.24010150986605341
Model Recall: 0.21756479144801735


Test on low-quality, high-noise data.

In [28]:
despiked_hn_spectra = remove_spikes(noisy_spectra_ratioed) # Final despike
hn_scores = compute_scores(despiked_hn_spectra, (models, fin), weights)

In [29]:
filtered_hn_preds, hn_preds, hn_pred_probs = filter_predictions(hn_scores, models[0].classes, merge_clays=False, kls_thr=(0.5,0.7))

In [30]:
class_report = get_classification_report(hn_preds, restricted_labels)
print(f"Model Accuracy: {class_report['accuracy']}")
print(f"Model F1 Score: {class_report['macro avg']['f1-score']}")
print(f"Model Precision: {class_report['macro avg']['precision']}")
print(f"Model Recall: {class_report['macro avg']['recall']}")

Model Accuracy: 0.0030306828151169547
Model F1 Score: 0.0025067192702921343
Model Precision: 0.045539534620984805
Model Recall: 0.007548758288177101
